# 👋 Hi there, hope you safe and well. This is for a general simulation procedure for any RCSB PDB structures containing ligand, with a special focous on small molecules. 

This notebook take http://zarbi.chem.yale.edu/ligpargen/namd_tutorial.html as a reference. This notebook is wrritten by quantaosun@gmail.com in Shanghai, China, 2021.


# The tutorial uses an example structure of T4 Lysozyme L99A with a Benzene molecule bound (PDB ID: 4W52.pdb)

**The overall procedure is as below**

0.   Install Anaconda
1.   Install pymol, vmd, and openmm, pdbfixer
2.   User input a PDB ID, pdbfixer fix and prepare it
3.   separate ligand and protein
4.   generate topology for ligand using LigParGen server
5.   combine psf and pdb of protein and ligand, vmd prepare NAMD input files.
6.   generate conf file for NAMD simulation. 
7.   use GPU accelerted NAMD simulate the job




# Now Switch to GPU

-------------------------------------------------------------------------
When you see a 👋 symbol, that is where you need to click the play button.
When you see a ⚠️ or ⚠️⚠️， that is where eithter need you to do some extra work or a reminder it is easy to go wrong.
-------------------------------------------------------------------------

---



In [ ]:
#@title 👋 1.Install Dependencies, this could take a while.
!mkdir external-libraries
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /home/aistudio/external-libraries
#@title Install Pymol
! yes | /home/aistudio/external-libraries/bin/conda install -c conda-forge vmd
#@title Install VMD

!yes | /home/aistudio/external-libraries/bin/conda install -c conda-forge pymol-open-source
#install openmm
!yes | /home/aistudio/external-libraries/bin/conda install -c conda-forge openmm
#install pdbfixer
!yes | /home/aistudio/external-libraries/bin/conda install -c conda-forge pdbfixer
! /home/aistudio/external-libraries/bin/pip install py3Dmol
!yes | /home/aistudio/external-libraries/bin/conda install -c rdkit rdkit
#install csh shell for step 8 or step 9
!apt-get install csh
import sys
sys.path.append('/home/aistudio/external-libraries/lib/python3.7/')
import sys
sys.path.append('/home/aistudio/external-libraries/lib/python3.7/site-packages/')
#import rdkit for visualization purpose
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Chem import RDConfig
from rdkit.Chem import rdBase
from rdkit.Chem import MolFromPDBFile

In [ ]:

#@title 👋 2.Input a PDB id. This will download and prepare this PDB complex.

complex = "4W52" #@param {type:"string"}
pdb = complex + ".pdb"
!wget http://www.pdb.org/pdb/files/'{pdb}'
#!/home/aistudio/external-libraries/bin/pdbfixer  '{pdb}' --ph=7  --replace-nonstandard --add-residues 

In [5]:
#@title ⚠️Alternatively you can upload your own protein structures from docking. In this case, you are suggested to use your local pymol instead of running the next cell.

In [ ]:
#@title 👋 3.Spilit protien and ligand, capital letters only.
pdb_id = "4W52" #@param {type:"string"}
ligand_name = "BNZ" #@param {type:"string"}
#create ligand= resn H3E 
#create active_water= resn HOH within 5 of ligand
#remove resn HOH &! active_water
com_file = open('prepare.pml','w')
com_file.write('''
load 4W52.pdb
remove resn HOH
h_add elem O or elem N
select 4W52-BNZ, resn BNZ 
select 4W52-receptor, 4W52 and not 4W52-BNZ
save 4W52-BNZ.pdb, 4W52-BNZ
save 4W52-receptor.pdb, 4W52-receptor    
''')
com_file.close()
!sed -i "s/4W52/$pdb_id/g" prepare.pml
!sed -i "s/BNZ/$ligand_name/g" prepare.pml
#@title 3.Spilit complex to protein and ligand
%cd /content/
! /home/aistudio/external-libraries/bin/pymol -c prepare.pml
#for the sake of LigParGen, we need add H atom to ligand

# 👋 ⚠️4.Download ligand, Uplaod it (Ligand should be added with H atoms before submitting.) to LigParGen, download the generated ZIP file.  Upload the *zip file here.
http://zarbi.chem.yale.edu/ligpargen/index.html 

In [6]:
#@title 👋 ⚠️ 5. Combine and Manupilate Structures. remove any line with TER and the connectivity information. Moreover, change the residue number of Benzene ligand for continuity according to the last residue number of the protein, and add a ⚠ Chain name like ⚠️a capital B to the ligand if there is no one. then upload the complex_fixed.pdb here.
Topology_ZIP_file_name_from_LigParGen = "unk" #@param {type:"string"}
protein_name = "5YVE-receptor" #@param {type:"string"}
ligand_name = "unk" #@param {type:"string"}

zip = Topology_ZIP_file_name_from_LigParGen + ".zip"
protein = protein_name + ".pdb"
ligand = ligand_name + ".pdb"
%cd /content
#!unzip $zip
#!cp tmp/*.pdb ./
#!cp tmp/*.prm ./
#!cp tmp/*.rtf ./
!cat $protein  $ligand > complex.pdb

/content


In [7]:
#@title  👋 ⚠️6. Generate ionized.pdb and ionized.psf with VMD by filling the RTF name download from LigParGen. Alternatively, you could refer to http://zarbi.chem.yale.edu/ligpargen/namd_tutorial.html 
RTF_name = "unk" #@param {type:"string"}
#Download opls amino acid topology file.
!wget http://traken.chem.yale.edu/doc/top_opls_aam.inp
com_file = open('psfgen.pgn','w')
com_file.write('''
package require psfgen   
topology top_opls_aam.inp
topology BNZ.rtf 

pdbalias HIS HSD
pdbalias atom SER HG HG1
pdbalias residue HIS HSE     
pdbalias atom ILE CD1 CD

segment A {pdb complex.pdb}  
coordpdb complex.pdb A   
guesscoord   
writepdb complex_autopsf.pdb     
writepsf complex_autopsf.psf 

package require solvate  
solvate complex_autopsf.psf complex_autopsf.pdb -t 5 -o complex_wb 
package require autoionize
autoionize -psf complex_wb.psf -pdb complex_wb.pdb -neutralize -o ionized
set ubq [atomselect top all]
measure minmax $ubq  
measure center $ubq   
exit    
''')
com_file.close()
!sed -i "s/BNZ/{RTF_name}/g" psfgen.pgn
!/home/aistudio/external-libraries/bin/vmd -dispdev text -e psfgen.pgn

--2021-10-17 03:46:02--  http://traken.chem.yale.edu/doc/top_opls_aam.inp
Resolving traken.chem.yale.edu (traken.chem.yale.edu)... 130.132.25.200
Connecting to traken.chem.yale.edu (traken.chem.yale.edu)|130.132.25.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70933 (69K) [text/plain]
Saving to: ‘top_opls_aam.inp’

top_opls_aam.inp    100%[===================>]  69.27K   278KB/s    in 0.2s    

2021-10-17 03:46:03 (278 KB/s) - ‘top_opls_aam.inp’ saved [70933/70933]

/home/aistudio/external-libraries/lib/vmd_LINUXAMD64: /usr/lib64-nvidia/libGL.so.1: no version information available (required by /home/aistudio/external-libraries/lib/vmd_LINUXAMD64)
Info) VMD for LINUXAMD64, version 1.9.3 (November 30, 2016)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K.,

⚠️7. Download and modify the NAMD conf files, change periodic vectors in minimisation and nvt conf. Change parameter file path and simulation steps if necessary


```
# ⚠️Use the command here to measure the water box size, then put the dimension vector to the PBC section of the conf files⚠️
set all [atomselect top all] 
set mm [measure minmax $all] # use the DIMENSION between MIN and MAX for CELL BASIS VECTOR DIMENSION
measure center $all # Use the CENTER for CELLBASIS ORIGIN

```



In [ ]:
#@title Mount to Goole Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 👋 7.Prepare NAMD software.
NAMD_tarball_PATH = "/content/drive/MyDrive/softwares/NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA-SingleNode.tar.gz" #@param {type:"string"}

!tar -xvf '{NAMD_tarball_PATH}'


In [ ]:
#@title 👋 ⚠️8.Download NAMD confiles and then manually modify it acoording to current input files.
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/step4_equilibration.inp
!wget  https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/step5_production.inp
!cp *.namd simulation
%cd simulation
!

/content/simulation


It is suggested that you run the simulation locally first to verify every thing is properly set up.

In [ ]:
#@title 👋 9.Run the Simulation and Copy result back to Goolge Drive every 1 ns, by default this is a 10-ns simulation
#%cd /content/simulation
#!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/run.csh
!csh run.csh



# If your encounter some issue, please report it on github or write me an email quantaosun@gmail.com ヾ(￣▽￣)Bye~Bye~